In [5]:
import os
import json
import numpy as np
from PIL import Image, ImageDraw
import pandas as pd
import glob

from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [6]:
def load_json(json_path):
    with open(json_path, 'r', encoding='utf-8') as file:
        return json.load(file)

In [ ]:
def create_mask(image_shape, polygons):
    mask = Image.new('L', image_shape, 0)
    for polygon in polygons:
        xy = [(point['x1'], point['y1']) for point in polygon['location']]
        ImageDraw.Draw(mask).polygon(xy, outline=1, fill=1)
    return mask

In [ ]:
base_folder = '무증상'
class_folders = os.listdir(base_folder)

image_paths = []
json_paths = []

for class_folder in class_folders:
    image_files = glob.glob(os.path.join(base_folder, class_folder, '*.jpg'))
    json_files = [file.replace('.jpg', '.json') for file in image_files]
    
    image_paths.extend(image_files)
    json_paths.extend(json_files)

In [ ]:

for img_path, json_path in zip(image_paths, json_paths):
    json_data = load_json(json_path)
    image = Image.open(img_path)
    image_shape = image.size

    # breed와 label 추출
    breed = json_data['metaData']['breed']
    diagnosis = json_data['metaData']['diagnosis']  # 예시로 diagnosis를 사용

    # 폴리곤 좌표를 이용하여 마스크 생성
    polygons = [info for info in json_data['labelingInfo'] if info['type'] == 'polygon']
    mask = create_mask(image_shape, polygons)

    # 마스크 이미지 저장 경로 설정
    mask_image_path = img_path.replace('.png', '_mask.png')
    mask.save(mask_image_path)

    # 데이터프레임에 정보 추가
    data.append({
        'image_path': img_path,
        'mask_path': mask_image_path,
        'breed': breed,
        'diagnosis': diagnosis
    })

# 데이터프레임 생성 및 CSV 파일로 저장
df = pd.DataFrame(data)
df.to_csv('dataset.csv', index=False)

In [ ]:
# 컨볼루션 블록 생성
def conv_block(input_tensor, num_filters):    
    x = Conv2D(num_filters, (3, 3), activation='relu', padding='same')(input_tensor)
    x = Conv2D(num_filters, (3, 3), activation='relu', padding='same')(x)
    return x

# 인코더 블록 생성
def encoder_block(input_tensor, num_filters):    
    x = conv_block(input_tensor, num_filters)
    p = MaxPooling2D((2, 2))(x)
    return x, p

# 디코더 블록 생성
def decoder_block(input_tensor, concat_tensor, num_filters):    
    x = UpSampling2D((2, 2))(input_tensor)
    x = concatenate([x, concat_tensor], axis=-1)
    x = conv_block(x, num_filters)
    return x

def unet(input_size=(256, 256, 1), num_filters_start=64, num_blocks=4, num_classes=7):
    inputs = Input(input_size)
    x = inputs
    skips = []
    
    for i in range(num_blocks):
        x, x_pool = encoder_block(x, num_filters_start * (2 ** i))
        skips.append(x)
        x = x_pool
        
    x = conv_block(x, num_filters_start * (2 ** num_blocks))
    
    for i in reversed(range(num_blocks)):
        x = decoder_block(x, skips[i], num_filters_start * (2 ** i))

    outputs = Conv2D(num_classes, (1, 1), activation='softmax')(x)

    model = Model(inputs=[inputs], outputs=[outputs])
    return model


In [ ]:
model = unet()
model.summary()

In [ ]:
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

X = # 이미지 데이터 로드
y = # 마스크 데이터 로드

# 데이터를 훈련 세트와 검증 세트로 분할합니다.
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1)

# 모델을 학습합니다.
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=2)

# 모델 성능을 평가합니다.
model.evaluate(X_val, y_val)